In [ ]:
!pip install keras-tcn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 26.1 MB/s eta 0:00:00


In [ ]:
!pip install neuralforecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 124.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader
from numpy.random import seed
from random import randrange
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from numpy.random import seed

In [ ]:
df = pd.read_excel("S&P 500 Realized Volatility.xlsx",index_col='Date',parse_dates=True)
df1 = pd.read_excel("S&P 500 Realized Volatility (NBEATSx).xlsx",index_col='Date',parse_dates=True)
df1.reset_index(inplace=True)
df1.rename(columns={'Date':'ds'}, inplace=True)
df1.rename(columns={'Realised':'y'}, inplace=True)
df1=df1[:len(df1["y"])-1]
test_length=len(df1["y"])-int(len(df1['y'])*0.7)

In [ ]:
def df_to_X_y(window_size,test,scaled_test_input):
  X=[]
  y=[]
  for i in range(int(len(test)/3)):
    row = [[a] for a in scaled_test_input[i*3:(i*3)+window_size]]
    X.append(row)
  return np.array(X)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

In [ ]:
scaler = MinMaxScaler()

In [ ]:
df=df[:len(df)-1]

# H=3 Forecasts

In [ ]:
#LSTM
n_features = 2
n_input = 189
train = df.iloc[:int(len(df['Realised'])*0.7)]
test = df.iloc[int(len(df['Realised'])*0.7)-n_input:]
scaler.fit(train)
scaled_train_input = scaler.transform(train)
scaled_test_input = scaler.transform(test)
train = df["Realised"].iloc[:int(len(df['Realised'])*0.7)]
test = df["Realised"].iloc[int(len(df['Realised'])*0.7):]
train = train.array.reshape(-1, 1)
test = test.array.reshape(-1, 1)
scaler.fit(train)
scaled_train_target = scaler.transform(train)
scaled_test_target = scaler.transform(test)
generator = TimeseriesGenerator(scaled_train_input, scaled_train_target, length=n_input, batch_size=32)
model = Sequential()
model.add(LSTM(56, activation='tanh', input_shape=(n_input, n_features)))
model.add(Dense(3))
model.compile(optimizer='adam', loss='mse', metrics=["accuracy", "mean_absolute_error"])
model.fit(generator,epochs=7)
test=test[:len(test)-len(test)%3]
X = df_to_X_y(n_input,test,scaled_test_input)
X=X.reshape(int(((len(test)/3))), n_input, n_features)
y_hat=model.predict(X)
LSTM_true_predictions = scaler.inverse_transform(y_hat)

#TCN
n_features = 2
n_input = 126
train = df.iloc[:int(len(df['Realised'])*0.7)]
test = df.iloc[int(len(df['Realised'])*0.7)-n_input:]
scaler.fit(train)
scaled_train_input = scaler.transform(train)
scaled_test_input = scaler.transform(test)
train = df["Realised"].iloc[:int(len(df['Realised'])*0.7)]
test = df["Realised"].iloc[int(len(df['Realised'])*0.7):]
train = train.array.reshape(-1, 1)
test = test.array.reshape(-1, 1)
scaler.fit(train)
scaled_train_target = scaler.transform(train)
scaled_test_target = scaler.transform(test)
generator = TimeseriesGenerator(scaled_train_input, scaled_train_target, length=n_input, batch_size=32)
model = Sequential()
model.add(TCN(kernel_size=4, dilations=[1,2,2,2,1], nb_stacks=1,padding='causal',
              dropout_rate=0.2,activation='relu',kernel_initializer='he_normal', input_shape=(n_input, n_features)))
model.add(Dense(3))
model.compile(optimizer='adam', loss='huber_loss', metrics=["accuracy", "mean_absolute_error"])
model.fit(generator,epochs=15)
test=test[:len(test)-len(test)%3]
X = df_to_X_y(n_input,test,scaled_test_input)
X=X.reshape(int(((len(test)/3))), n_input, n_features)
y_hat=model.predict(X)
TCN_true_predictions = scaler.inverse_transform(y_hat)

model = NBEATSx(h=3, input_size=21,
                  loss=MQLoss(level=[90]),
                  scaler_type='minmax',
                  learning_rate= 0.0001,
                  stack_types=['identity','identity'],
                  n_blocks=[1,1],
                  mlp_units= [[512, 512], [512, 512]],
                  windows_batch_size=60,
                  num_lr_decays=5,
                  val_check_steps=100,
                  n_harmonics=0, n_polynomials=0,
                  hist_exog_list=['VIX'],
                  max_steps=150,
                  early_stop_patience_steps=1,
                  random_seed=randrange(129228148)
                  )
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=9,static_df=AirPassengersStatic,n_windows=None, test_size=test_length-test_length%3,step_size=3)
forecasts = forecasts.dropna()
if "NBEATSx-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["NBEATSx"].values
else:
  Y_hat=forecasts["NBEATSx-median"].values
Y_true=forecasts["y"].values
NBEATSx_true_predictions = Y_hat

Epoch 1/7
122/122 [==============================] - 5s 13ms/step - loss: 0.0048 - accuracy: 0.0876 - mean_absolute_error: 0.0432
Epoch 2/7
122/122 [==============================] - 1s 12ms/step - loss: 0.0032 - accuracy: 0.1466 - mean_absolute_error: 0.0365
Epoch 3/7
122/122 [==============================] - 1s 8ms/step - loss: 0.0030 - accuracy: 0.2152 - mean_absolute_error: 0.0352
Epoch 4/7
122/122 [==============================] - 1s 8ms/step - loss: 0.0029 - accuracy: 0.1904 - mean_absolute_error: 0.0349
Epoch 5/7
122/122 [==============================] - 1s 8ms/step - loss: 0.0030 - accuracy: 0.1476 - mean_absolute_error: 0.0351
Epoch 6/7
122/122 [==============================] - 1s 8ms/step - loss: 0.0029 - accuracy: 0.1735 - mean_absolute_error: 0.0347
Epoch 7/7
19/19 [==============================] - 0s 4ms/step
Epoch 1/15
124/124 [==============================] - 8s 14ms/step - loss: 0.2020 - accuracy: 0.2874 - mean_absolute_error: 0.3547
Epoch 2/15
124/124 [==========

INFO:lightning_fabric.utilities.seed:Global seed set to 11156906


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [ ]:
df_forecasts=pd.read_excel("H=3.xlsx",index_col='Date',parse_dates=True,sheet_name="S&P500")

In [ ]:
LSTM_true_predictions=LSTM_true_predictions.reshape(len(test),1)
TCN_true_predictions=TCN_true_predictions.reshape(len(test),1)

In [ ]:
HAR_residuals=df_forecasts["Actuals"]-df_forecasts["HAR"]
GARCH_residuals=df_forecasts["Actuals"]-df_forecasts["GARCH"]
DJR_GARCH_residuals=df_forecasts["Actuals"]-df_forecasts["DJR-GARCH"]
LSTM_residuals=test-LSTM_true_predictions
TCN_residuals=test-TCN_true_predictions
NBEATSx_residuals=Y_true-NBEATSx_true_predictions

In [ ]:
!pip install EntropyHub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [ ]:
from scipy.stats import anderson
from EntropyHub import PermEn
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.stats.diagnostic as dg

# Normality Test (Anderson-Darling Test)

In [ ]:
LSTM_residuals=LSTM_residuals.reshape(len(LSTM_residuals))
TCN_residuals=TCN_residuals.reshape(len(TCN_residuals))

In [ ]:
print(f"LSTM: {anderson(LSTM_residuals)}")
print(" ")
print(f"TCN: {anderson(TCN_residuals)}")
print(" ")
print(f"NBEATSx: {anderson(NBEATSx_residuals)}")
print(" ")
print(f"HAR: {anderson(HAR_residuals)}")
print(" ")
print(f"GARCH: {anderson(GARCH_residuals)}")
print(" ")
print(f"DJR-GARCH: {anderson(DJR_GARCH_residuals)}")

LSTM: AndersonResult(statistic=69.28993388850677, critical_values=array([0.575, 0.655, 0.785, 0.916, 1.09 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.0006305057009285316, scale=0.004286307526215635)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
 
TCN: AndersonResult(statistic=59.185288112556464, critical_values=array([0.575, 0.655, 0.785, 0.916, 1.09 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=-5.74510958023237e-05, scale=0.004032337900306307)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
 
NBEATSx: AndersonResult(statistic=45.453862755665796, critical_values=array([0.575, 0.655, 0.785, 0.916, 1.09 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.00016591210428657262, scale=0.004425726989909321)
 success: True
 message: '`anderson` successfully fit the

# Stationarity Test (Augmented Dickey Fuller Test)

In [ ]:
print(f"LSTM: {adfuller(LSTM_residuals)}")
print(" ")
print(f"TCN: {adfuller(TCN_residuals)}")
print(" ")
print(f"NBEATSx: {adfuller(NBEATSx_residuals)}")
print(" ")
print(f"HAR: {adfuller(HAR_residuals)}")
print(" ")
print(f"GARCH: {adfuller(GARCH_residuals)}")
print(" ")
print(f"DJR-GARCH: {adfuller(DJR_GARCH_residuals)}")

LSTM: (-8.026019584623683, 2.0164224614206915e-12, 17, 1737, {'1%': -3.434120287918905, '5%': -2.8632053717943005, '10%': -2.5676565959447415}, -14236.46617895335)
 
TCN: (-7.9893728869081, 2.4983355850356867e-12, 17, 1737, {'1%': -3.434120287918905, '5%': -2.8632053717943005, '10%': -2.5676565959447415}, -14283.305494447093)
 
NBEATSx: (-17.13888765939098, 7.066996119855161e-30, 7, 1740, {'1%': -3.4341137777946718, '5%': -2.8632024980220288, '10%': -2.56765506572863}, -13996.779566055051)
 
HAR: (-17.698456690068603, 3.545882940917726e-30, 4, 1751, {'1%': -3.434090098474972, '5%': -2.8631920451777195, '10%': -2.5676494998436072}, -14061.669421083476)
 
GARCH: (-9.222610031310433, 1.763354112414457e-15, 16, 1739, {'1%': -3.4341159453366448, '5%': -2.863203454843483, '10%': -2.5676555752134256}, -13685.613445648913)
 
DJR-GARCH: (-9.438798157671368, 4.958738401590965e-16, 13, 1742, {'1%': -3.4341094501874854, '5%': -2.8632005876775297, '10%': -2.567654048515701}, -13784.27662892042)


# Information Entropy Test (Permutation Entropy)

In [ ]:
cPE=PermEn(LSTM_residuals, m = 2, tau = 1, Typex = "none", tpx = -1, Logx= 2, Norm = False)[2]
print(f"LSTM: {cPE}")
print(" ")
cPE=PermEn(TCN_residuals, m = 2, tau = 1, Typex = "none", tpx = -1, Logx= 2, Norm = False)[2]
print(f"TCN: {cPE}")
print(" ")
cPE=PermEn(NBEATSx_residuals, m = 2, tau = 1, Typex = "none", tpx = -1, Logx= 2, Norm = False)[2]
print(f"NBEATSx: {cPE}")
print(" ")
cPE=PermEn(HAR_residuals, m = 2, tau = 1, Typex = "none", tpx = -1, Logx= 2, Norm = False)[2]
print(f"HAR: {cPE}")
cPE=PermEn(GARCH_residuals, m = 2, tau = 1, Typex = "none", tpx = -1, Logx= 2, Norm = False)[2]
print(f"GARCH: {cPE}")
cPE=PermEn(DJR_GARCH_residuals, m = 2, tau = 1, Typex = "none", tpx = -1, Logx= 2, Norm = False)[2]
print(f"DJR-GARCH: {cPE}")

LSTM: [0.99984149]
 
TCN: [0.99995404]
 
NBEATSx: [0.99989577]
 
HAR: [0.99993231]
GARCH: [0.99939076]
DJR-GARCH: [0.99901027]


# Diebold-Mariano Test

In [ ]:
df_for_date=df[int(len(df['Realised'])*0.7):]

In [ ]:
df_for_date=df_for_date.reset_index()

In [ ]:
test=test.reshape(len(LSTM_true_predictions))
TCN_true_predictions=TCN_true_predictions.reshape(len(TCN_true_predictions))
LSTM_true_predictions=LSTM_true_predictions.reshape(len(LSTM_true_predictions))

In [ ]:
data={'NBEATSx':NBEATSx_true_predictions, 'ds':forecasts['ds']}

NBEATSx_Forecasts=pd.DataFrame(data)

data={'LSTM':LSTM_true_predictions[:len(test)], 'TCN':TCN_true_predictions[:len(test)], 'Date':df_for_date["Date"]}

LSTM_TCN_Forecasts=pd.DataFrame(data)

In [ ]:
NBEATSx_Forecasts.set_index('ds', inplace=True)
LSTM_TCN_Forecasts.set_index('Date', inplace=True)

In [ ]:
df_merge =df_forecasts.merge(LSTM_TCN_Forecasts, left_index=True, right_index=True)
df_merge = NBEATSx_Forecasts.merge(df_merge, left_index=True, right_index=True)

In [ ]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return p_value

In [ ]:
df =df_merge
a=dm_test(df["Actuals"], df["NBEATSx"], df["LSTM"], crit="MAPE")
b=dm_test(df["Actuals"], df["NBEATSx"], df["TCN"], crit="MAPE")
c=dm_test(df["Actuals"], df["NBEATSx"], df["HAR"], crit="MAPE")
d=dm_test(df["Actuals"], df["TCN"], df["HAR"], crit="MAPE")
e=dm_test(df["Actuals"], df["LSTM"], df["HAR"], crit="MAPE")
f=dm_test(df["Actuals"], df["NBEATSx"], df["GARCH"], crit="MAPE")
g=dm_test(df["Actuals"], df["NBEATSx"], df["DJR-GARCH"], crit="MAPE")
h=dm_test(df["Actuals"], df["LSTM"], df["GARCH"], crit="MAPE")
i=dm_test(df["Actuals"], df["LSTM"], df["DJR-GARCH"], crit="MAPE")
j=dm_test(df["Actuals"], df["TCN"], df["GARCH"], crit="MAPE")
k=dm_test(df["Actuals"], df["TCN"], df["DJR-GARCH"], crit="MAPE")
l=dm_test(df["Actuals"], df["HAR"], df["GARCH"], crit="MAPE")
m=dm_test(df["Actuals"], df["HAR"], df["DJR-GARCH"], crit="MAPE")
n=dm_test(df["Actuals"], df["GARCH"], df["DJR-GARCH"], crit="MAPE")
o=dm_test(df["Actuals"], df["LSTM"], df["TCN"], crit="MAPE")


print(f"NBEATSx vs LSTM: {a}")
print(f"NBEATSx vs TCN: {b}")
print(f"NBEATSx vs HAR: {c}")
print(f"NBEATSx vs GARCH: {f}")
print(f"NBEATSx vs DJR-GARCH: {g}")
print(f"TCN vs HAR: {d}")
print(f"TCN vs GARCH: {j}")
print(f"TCN vs DJR-GARCH: {k}")
print(f"LSTM vs TCN: {o}")
print(f"LSTM vs HAR: {e}")
print(f"LSTM vs GARCH: {h}")
print(f"LSTM vs DJR-GARCH: {i}")
print(f"HAR vs GARCH: {l}")
print(f"HAR vs DJR-GARCH: {m}")
print(f"GARCH vs DJR-GARCH: {n}")

NBEATSx vs LSTM: 0.00696769558904669
NBEATSx vs TCN: 2.202419993741532e-23
NBEATSx vs HAR: 0.00741450339025911
NBEATSx vs GARCH: 0.0027618885307016073
NBEATSx vs DJR-GARCH: 0.002379276613416454
TCN vs HAR: 1.5068269090003923e-75
TCN vs GARCH: 1.0218166288309706e-10
TCN vs DJR-GARCH: 1.626287383936336e-17
LSTM vs TCN: 0.5976278443227173
LSTM vs HAR: 2.4122017794160505e-26
LSTM vs GARCH: 0.00903163628837573
LSTM vs DJR-GARCH: 0.0031808255557402
HAR vs GARCH: 0.09662452758358404
HAR vs DJR-GARCH: 0.2441634150251279
GARCH vs DJR-GARCH: 0.3666895968492977
